In [2]:
import tensorflow as tf #导入tensorflow
from tensorflow.examples.tutorials.mnist import input_data #导入手写数字数据集

import numpy as np #导入numpy
import matplotlib.pyplot as plt #plt是绘图工具，在训练过程中用于输出可视化结果
import matplotlib.gridspec as gridspec #gridspec是图片排列工具，在训练过程中用于输出可视化结果
import os

In [3]:
def save(saver, sess, logdir, step):
    """用于保存模型"""
    model_name = 'GAN_model'
    # 模型的保存路径为"logdir + GAN_model"
    checkpoint_path = os.path.join(logdir,model_name)
    # 保存模型
    saver.save(sess, checkpoint_path, global_step = step)
    print("the checkpoint has been created")

def xavier_init(size):
    """初始化参数时使用xavier_init"""
    in_dim = size[0]
    # 初始化标准差
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    # 返回初始化的结果
    return tf.random_normal(shape=size, stddev=xavier_stddev)

# X表示真的样本(即真实的手写数字)
X = tf.placeholder(tf.float32, shape=[None, 784])

#表示使用xavier方式初始化的判别器的D_W1参数，是一个784行128列的矩阵
D_W1 = tf.Variable(xavier_init([784, 128]))
#表示全零方式初始化的判别器的D_1参数，是一个长度为128的向量
D_b1 = tf.Variable(tf.zeros(shape=[128]))
D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

#theta_D 表示判别器的可训练参数集合
theta_D = [D_W1, D_W2, D_b1, D_b2]

# Z表示生成器的输入(在这里是噪声)，是一个N列100行的矩阵
Z = tf.placeholder(tf.float32, shape=[None, 100])

# 表示使用xavier方式初始化的生成器的G_W1参数，是一个100行128列的矩阵
G_W1 = tf.Variable(xavier_init([100, 128]))
# 表示全零方式初始化的生成器的G_b1参数，是一个长度为128的向量
G_b1 = tf.Variable(tf.zeros(shape=[128]))
G_W2 = tf.Variable(xavier_init([128, 784])) 
G_b2 = tf.Variable(tf.zeros(shape=[784]))

#theta_G表示生成器的可训练参数集合
theta_G = [G_W1, G_W2, G_b1, G_b2] 

In [4]:
def sample_Z(m, n):
    """ 生成维度为[m, n]的随机噪声作为生成器G的输入"""
    return np.random.uniform(-1., 1., size=[m, n])

def generator(z): 
    """
    # 生成器，z的维度为[N, 100]
    # 输入的随机噪声乘以G_W1矩阵加上偏置G_b1，G_h1维度为[N, 128]
    # G_h1乘以G_W2矩阵加上偏置G_b2，G_log_prob维度为[N, 784]
    # G_log_prob经过一个sigmoid函数，G_prob维度为[N, 784]
    """
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob #返回G_prob

def discriminator(x): 
    """
    #判别器，x的维度为[N, 784]
    # 输入乘以D_W1矩阵加上偏置D_b1，D_h1维度为[N, 128]
    # D_h1乘以D_W2矩阵加上偏置D_b2，D_logit维度为[N, 1]
    # D_logit经过一个sigmoid函数，D_prob维度为[N, 1]
    """
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1) 
    D_logit = tf.matmul(D_h1, D_W2) + D_b2 
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob, D_logit #返回D_prob, D_logit

def plot(samples):
    """
    #保存图片时使用的plot函数
    #初始化一个4行4列包含16张子图像的图片
    #调整子图的位置
    #置子图间的间距
    #依次将16张子图填充进需要保存的图像
    """
    fig = plt.figure(figsize=(4, 4)) 
    gs = gridspec.GridSpec(4, 4) 
    gs.update(wspace=0.05, hspace=0.05) 
 
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
 
    return fig

In [5]:
G_sample = generator(Z) #取得生成器的生成结果
D_real, D_logit_real = discriminator(X) #取得判别器判别的真实手写数字的结果
D_fake, D_logit_fake = discriminator(G_sample) #取得判别器判别的生成的手写数字的结果
 
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real))) #对判别器对真实样本的判别结果计算误差(将结果与1比较)
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake))) #对判别器对虚假样本(即生成器生成的手写数字)的判别结果计算误差(将结果与0比较)
D_loss = D_loss_real + D_loss_fake #判别器的误差
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake))) #生成器的误差(将判别器返回的对虚假样本的判别结果与1比较)
 
dreal_loss_sum = tf.summary.scalar("dreal_loss", D_loss_real) #记录判别器判别真实样本的误差
dfake_loss_sum = tf.summary.scalar("dfake_loss", D_loss_fake) #记录判别器判别虚假样本的误差
d_loss_sum = tf.summary.scalar("d_loss", D_loss) #记录判别器的误差
g_loss_sum = tf.summary.scalar("g_loss", G_loss) #记录生成器的误差
 
summary_writer = tf.summary.FileWriter('./logs/snapshots/', graph=tf.get_default_graph()) #日志记录器
 
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D) #判别器的训练器
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G) #生成器的训练器
 
mb_size = 128 #训练的batch_size
Z_dim = 100 #生成器输入的随机噪声的列的维度
 
mnist = input_data.read_data_sets('./MNIST_data', one_hot=True) #mnist是手写数字数据集

sess = tf.Session() #会话层
sess.run(tf.global_variables_initializer()) #初始化所有可训练参数
 
if not os.path.exists('./logs/out/'): #初始化训练过程中的可视化结果的输出文件夹
    os.makedirs('./logs/out/')
 
if not os.path.exists('./logs/snapshots/'): #初始化训练过程中的模型保存文件夹
    os.makedirs('./logs/snapshots/')

saver = tf.train.Saver(var_list=tf.global_variables(), max_to_keep=50) #模型的保存器
 
i = 0 #训练过程中保存的可视化结果的索引
 
for it in range(1000000): #训练100万次
    if it % 1000 == 0: #每训练1000次就保存一下结果
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})
 
        fig = plot(samples) #通过plot函数生成可视化结果
        plt.savefig('./logs/out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight') #保存可视化结果
        i += 1
        plt.close(fig)
 
    X_mb, _ = mnist.train.next_batch(mb_size) #得到训练一个batch所需的真实手写数字(作为判别器的输入)
 
    #下面是得到训练一次的结果，通过sess来run出来
    _, D_loss_curr, dreal_loss_sum_value, dfake_loss_sum_value, d_loss_sum_value = sess.run([D_solver, D_loss, dreal_loss_sum, dfake_loss_sum, d_loss_sum], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr, g_loss_sum_value = sess.run([G_solver, G_loss, g_loss_sum], feed_dict={Z: sample_Z(mb_size, Z_dim)})
 
    if it%100 ==0: #每过100次记录一下日志，可以通过tensorboard查看
        summary_writer.add_summary(dreal_loss_sum_value, it)
        summary_writer.add_summary(dfake_loss_sum_value, it)
        summary_writer.add_summary(d_loss_sum_value, it)
        summary_writer.add_summary(g_loss_sum_value, it)
 
    if it % 1000 == 0: #每训练1000次输出一下结果
        save(saver, sess, './logs/snapshots/', it)
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
the checkpoint has been created
Iter: 0
D loss: 1.477
G_loss: 2.653
()
the checkpoint has been created
Iter: 1000
D loss: 0.003478
G_loss: 8.079
()
the checkpoint has been created
Iter: 2000
D loss: 0.0337
G_loss: 5.459
()
the checkpoint has been created
Iter: 3000
D loss: 0.02842
G_loss: 5.474
()
the checkpoint has been created
Iter: 4000
D loss: 0.1003
G_loss: 5.634
()
the checkpoint has been created
Iter: 5000
D loss: 0.1633
G_loss: 5.117
()
the checkpoint has been created
Iter: 6000
D loss: 0.4167
G_loss: 4.639
()
the checkpoint has been created
Iter: 7000
D loss: 0.4705
G_loss: 4.726
()
the checkpoint has been created
Iter: 8000
D loss: 0.4253
G_loss: 3.835
()
the checkpoint has been created
Iter: 9000
D loss: 0.4615
G_loss: 3.445
()
the checkpoint has been created
Iter: 10000
D los

KeyboardInterrupt: 